In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchsummary import summary
#from scipy.spatial.distance import canberra, euclidean

#from torchpairwise import canberra_distances

import my_utility as mu

In [ ]:
# Parametri della rete
epochs = 10
batch_size = 16
learning_rate = 0.01
margin = 1  # Margin for contrastive loss.

In [ ]:
# Selezione del device da usare per il training
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

In [ ]:
n = 384 # Dimensione codifica frattale

# Parametri del dataset
num_samples_train = 8000
num_samples_val = 2000
num_samples_test = 2000

In [ ]:
root_dir = ".\img_celeba_10000"

all_img_dir = root_dir + "\all_img_celeba"
all_codify_dir = root_dir + "\all_codify_celeba\codify_celeba_all.csv"


all_set = mu.ImageDataSet(img_dir = all_img_dir, codify_dir = all_codify_dir)



all_loader = DataLoader(all_set, batch_size=batch_size, shuffle=True)

In [ ]:
model = mu.SiameseNeuralNetwork().to(device)
summary(model, input_size=(1, 128, 128))

In [ ]:
def train(model, optimizer, loss_fn, train_loader, epochs, device):
    for epoch in range(epochs):
        training_loss = 0.0
        valid_loss = 0.0
        
        # Training loop
        model.train()
        for batch in train_loader:
            inputs, label = batch
            inputs = inputs.to(device)
            label = label.to(device)

            # Calcolo dell'embedding di output
            output = model(inputs)
            
            # Confronto con l'embedding di input
            loss = loss_fn(output, label)
            loss.requires_grad = True
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            training_loss += loss.data.item()

        
        print(f"Epoch [{epoch+1}/{epochs}] ----> \nTraining loss: {training_loss:.4f} \n")
    


In [ ]:
# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Lr scheduler
scheduler = torch.optim.lr_scheduler.StepLR(optimizer=optimizer, step_size=10)

In [ ]:
distance = mu.custom_canberra_distance
criterion = mu.CustomLoss(distance=distance)

In [1]:
train(model=model, optimizer=optimizer, loss_fn=criterion, train_loader=all_loader, epochs=epochs, device=device)


In [ ]:
torch.save(model.state_dict(), f="model_all_images")